# Certainty annotations

In this Notebook: certainty classifiers from [Jiaxin Pei](https://github.com/Jiaxin-Pei/certainty-estimator/) are applied to a corpus of claims. The model can be easily installed using `pip install certainty-estimator`.

In [1]:
import pandas as pds
import pickle

from certainty_estimator.predict_certainty import CertaintyEstimator
from tqdm import tqdm

from utils.ClaimDB import ClaimDB
from utils.Paper import Paper
import json
from utils.Corpus import Corpus
from utils.utils import load_corpus_object, CustomEncoder

## Load the certainty estimators

In [2]:
sentence_estimator = CertaintyEstimator('sentence-level', cuda = True)
aspect_estimator = CertaintyEstimator('aspect-level', cuda = True)

/home/clbleuze/myenv/lib/python3.9/site-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/clbleuze/myenv/lib/python3.9/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/clbleuze/myenv/lib/python3.9/site-packages/transformers/modeling_utils.py:3027: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [13]:
sents = ["We have proven that our model compares similarly to BERT",
         "We leave this issue to future research.",
        "We obtain F1 score of more than 0.85.",
        "Maybe this is due to small dataset size"]

res_sentence = sentence_estimator.predict(sents, tqdm = tqdm)
print(type(res_sentence))
for res in res_sentence:
    print(res)

1it [00:00, 18.95it/s]

<class 'list'>
4.944007
4.6444902
4.6428776
4.1955776


In [4]:
res_aspect = aspect_estimator.predict(sents, tqdm = tqdm)
for res in res_aspect:
    print(res)

0it [00:00, ?it/s]/home/clbleuze/myenv/lib/python3.9/site-packages/transformers/modeling_utils.py:1052: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
1it [00:00,  2.09it/s]

[('Probability', 'Certain'), ('Framing', 'Certain')]
[('Suggestion', 'Certain'), ('Framing', 'Certain')]
[('Number', 'Certain'), ('Framing', 'Certain')]
[('Probability', 'Uncertain')]


## Load data to annotate

In [20]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    for paper in tqdm(corpus_ACL.papers):
    # collect the list of sentences to annotate in certainty
        sents = paper.content.sentence.values.tolist()
    
        # use the certainty models
        if "sentence_certainty" not in paper.content.columns:
            sent_certainty = sentence_estimator.predict(sents)
            paper.content["sentence_certainty"] = sent_certainty
        
        if "aspect_certainty" not in paper.content.columns:
            aspect_certainty = aspect_estimator.predict(sents)
            paper.content["aspect_certainty"] = aspect_certainty

 16%|█▌        | 6930/42759 [1:33:11<6:51:43,  1.45it/s] IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

 34%|███▍      | 14735/42759 [3:14:43<6:10:20,  1.26it/s] 


KeyboardInterrupt: 

In [23]:
corpus_ACL.papers[14001].content.sentence_certainty.describe()

count    305.000000
mean       4.831699
std        0.285639
min        2.867607
25%        4.819613
50%        4.937650
75%        4.978834
max        5.322650
Name: sentence_certainty, dtype: float64

In [30]:
for paper in corpus_ACL.papers:
    paper.corpus = corpus_ACL

for paper in corpus_ACL.papers_with_errors:
    paper.corpus = corpus_ACL

In [32]:
corpus_ACL_dict = corpus_ACL.to_dict()
corpus_ACL_s = json.dumps(corpus_ACL_dict, cls = CustomEncoder)
with open("data/test_drop_corpus_ACL.json", "w") as f:
    json.dump(corpus_ACL_s, f)

In [33]:
corpus_ACL_reloaded = load_corpus_object("data/test_drop_corpus_ACL.json")

In [41]:
corpus_ACL.

AttributeError: 'Corpus' object has no attribute 'path'